<a href="https://colab.research.google.com/github/sudhan670/Noise-Cancellation/blob/main/Noise_cancellation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy scipy librosa soundfile pydub

In [ ]:
pip install numpy librosa soundfile noisereduce torch scipy

In [3]:
pip install noisereduce

In [5]:
import numpy as np
import librosa
import soundfile as sf
import noisereduce as nr
from scipy.io import wavfile
import torch
from torch import nn
import os
import warnings
warnings.filterwarnings('ignore')

class AudioCleaner:
    def __init__(self):
        """Initialize the audio cleaning system with ML models"""
        self.sample_rate = 44100
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Initialize noise reduction model
        self.noise_reducer = self._create_noise_reducer()

    def _create_noise_reducer(self):
        """Create a simple neural network for noise reduction"""
        model = nn.Sequential(
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024)
        ).to(self.device)
        return model

    def clean_audio(self, input_path, output_path=None):
        """
        Clean audio file using ML-based noise reduction

        Args:
            input_path (str): Path to input audio file
            output_path (str): Path for output WAV file (optional)

        Returns:
            str: Path to cleaned audio file
        """
        try:
            # Generate output path if not provided
            if output_path is None:
                output_path = os.path.splitext(input_path)[0] + '_cleaned.wav'

            # Load and preprocess audio
            print("Loading audio file...")
            audio, sr = librosa.load(input_path, sr=self.sample_rate)

            # Apply initial noise reduction
            print("Applying ML-based noise reduction...")
            reduced_noise = nr.reduce_noise(
                y=audio,
                sr=sr,
                stationary=True,
                prop_decrease=0.75
            )

            # Apply additional cleaning using our neural network
            print("Applying deep learning enhancement...")
            cleaned_audio = self._apply_ml_cleaning(reduced_noise)

            # Normalize audio
            cleaned_audio = librosa.util.normalize(cleaned_audio)

            # Save cleaned audio
            print(f"Saving cleaned audio to {output_path}")
            sf.write(output_path, cleaned_audio, sr)

            return output_path

        except Exception as e:
            raise Exception(f"Error processing audio: {str(e)}")

    def _apply_ml_cleaning(self, audio):
        """Apply ML-based cleaning to audio segments"""
        # Process audio in chunks
        chunk_size = 1024
        cleaned_chunks = []

        for i in range(0, len(audio), chunk_size):
            chunk = audio[i:i + chunk_size]

            # Pad last chunk if needed
            if len(chunk) < chunk_size:
                chunk = np.pad(chunk, (0, chunk_size - len(chunk)))

            # Convert to tensor
            chunk_tensor = torch.FloatTensor(chunk).to(self.device)

            # Apply noise reduction model
            with torch.no_grad():
                cleaned_chunk = self.noise_reducer(chunk_tensor)

            cleaned_chunks.append(cleaned_chunk.cpu().numpy())

        # Concatenate chunks
        return np.concatenate(cleaned_chunks[:len(audio)//chunk_size * chunk_size])

def process_audio_file(input_file, output_file=None):
    """
    Process audio file with noise cleaning

    Args:
        input_file (str): Path to input audio file
        output_file (str): Path for output file (optional)

    Returns:
        str: Path to cleaned audio file
    """
    cleaner = AudioCleaner()
    return cleaner.clean_audio(input_file, output_file)

# Example usage
if __name__ == "__main__":
    # Example of how to use the audio cleaner
    input_path = "/content/countdown-from-10-105775.mp3"  # Replace with your input file
    output_path = "cleaned_audio.wav"  # Replace with desired output path

    try:
        cleaned_file = process_audio_file(input_path, output_path)
        print(f"Successfully cleaned audio file: {cleaned_file}")
    except Exception as e:
        print(f"Error: {str(e)}")

Loading audio file...
Applying ML-based noise reduction...
Applying deep learning enhancement...
Saving cleaned audio to cleaned_audio.wav
Successfully cleaned audio file: cleaned_audio.wav


In [7]:
import numpy as np
import librosa
import soundfile as sf
import noisereduce as nr
from scipy.io import wavfile
import torch
from torch import nn
import os
import warnings
warnings.filterwarnings('ignore')

class AudioCleaner:
    def __init__(self):
        """Initialize the audio cleaning system with ML models"""
        self.sample_rate = 44100
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")

        # Initialize noise reduction model
        self.noise_reducer = self._create_noise_reducer()

    def _create_noise_reducer(self):
        """Create a simple neural network for noise reduction"""
        model = nn.Sequential(
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024)
        ).to(self.device)
        return model

    def process_audio(self, input_path, clean_output_path=None, noise_output_path=None):
        """
        Process audio file using ML-based noise reduction and extract noise

        Args:
            input_path (str): Path to input audio file
            clean_output_path (str): Path for clean audio output (optional)
            noise_output_path (str): Path for noise audio output (optional)

        Returns:
            tuple: (clean_audio_path, noise_audio_path)
        """
        try:
            # Generate output paths if not provided
            if clean_output_path is None:
                clean_output_path = os.path.splitext(input_path)[0] + '_clean.wav'
            if noise_output_path is None:
                noise_output_path = os.path.splitext(input_path)[0] + '_noise.wav'

            # Load and preprocess audio
            print("Loading audio file...")
            audio, sr = librosa.load(input_path, sr=self.sample_rate)

            # First stage: Initial noise reduction
            print("Applying initial noise reduction...")
            reduced_noise = nr.reduce_noise(
                y=audio,
                sr=sr,
                stationary=True,
                prop_decrease=0.75
            )

            # Extract noise component
            print("Extracting noise component...")
            noise = audio - reduced_noise

            # Second stage: ML-based enhancement
            print("Applying deep learning enhancement...")
            cleaned_audio = self._apply_ml_cleaning(reduced_noise)

            # Normalize both signals
            cleaned_audio = librosa.util.normalize(cleaned_audio)
            noise = librosa.util.normalize(noise)

            # Save both clean audio and noise
            print(f"Saving clean audio to {clean_output_path}")
            sf.write(clean_output_path, cleaned_audio, sr)

            print(f"Saving noise audio to {noise_output_path}")
            sf.write(noise_output_path, noise, sr)

            return clean_output_path, noise_output_path

        except Exception as e:
            raise Exception(f"Error processing audio: {str(e)}")

    def _apply_ml_cleaning(self, audio):
        """Apply ML-based cleaning to audio segments"""
        # Process audio in chunks
        chunk_size = 1024
        cleaned_chunks = []
        total_chunks = len(audio) // chunk_size

        print(f"Processing {total_chunks} chunks...")

        for i in range(0, len(audio), chunk_size):
            chunk = audio[i:i + chunk_size]

            # Pad last chunk if needed
            if len(chunk) < chunk_size:
                chunk = np.pad(chunk, (0, chunk_size - len(chunk)))

            # Convert to tensor
            chunk_tensor = torch.FloatTensor(chunk).to(self.device)

            # Apply noise reduction model
            with torch.no_grad():
                cleaned_chunk = self.noise_reducer(chunk_tensor)

            cleaned_chunks.append(cleaned_chunk.cpu().numpy())

            # Print progress every 10%
            if (i // chunk_size) % (total_chunks // 10) == 0:
                print(f"Progress: {(i // chunk_size) * 100 // total_chunks}%")

        print("Progress: 100%")

        # Concatenate chunks
        return np.concatenate(cleaned_chunks[:len(audio)//chunk_size * chunk_size])

def process_audio_file(input_file, clean_output=None, noise_output=None):
    """
    Process audio file with noise cleaning and extraction

    Args:
        input_file (str): Path to input audio file
        clean_output (str): Path for clean audio output (optional)
        noise_output (str): Path for noise audio output (optional)

    Returns:
        tuple: (clean_audio_path, noise_audio_path)
    """
    cleaner = AudioCleaner()
    return cleaner.process_audio(input_file, clean_output, noise_output)

# Example usage
if __name__ == "__main__":
    # Allow command line input or use default path
    input_path = input("Enter the path to your audio file: ").strip('"\' ')

    if not os.path.exists(input_path):
        print(f"Error: File not found at {input_path}")
    else:
        try:
            print("\nProcessing audio file...")
            clean_file, noise_file = process_audio_file(input_path)
            print(f"\nProcessing complete!")
            print(f"Clean audio saved to: {noise_file}")
            print(f"Noise audio saved to: {clean_file}")
        except Exception as e:
            print(f"\nError: {str(e)}")

Enter the path to your audio file: /content/countdown-from-10-105775.mp3

Processing audio file...
Using device: cpu
Loading audio file...
Applying initial noise reduction...
Extracting noise component...
Applying deep learning enhancement...
Processing 516 chunks...
Progress: 0%
Progress: 9%
Progress: 19%
Progress: 29%
Progress: 39%
Progress: 49%
Progress: 59%
Progress: 69%
Progress: 79%
Progress: 88%
Progress: 98%
Progress: 100%
Saving clean audio to /content/countdown-from-10-105775_clean.wav
Saving noise audio to /content/countdown-from-10-105775_noise.wav

Processing complete!
Clean audio saved to: /content/countdown-from-10-105775_noise.wav
Noise audio saved to: /content/countdown-from-10-105775_clean.wav


In [ ]:
!pip install demucs


In [ ]:
!pip install torchaudio --upgrade
